# Code snippets for testing the functions



In [1]:
import pandas as pd

In [122]:
# Load the data in a dataframe structure
df = pd.read_excel('solventSelectionTool_table.xlsx', sheet = 0, header = 2)
# Drop first row as it is empty
df = df[1:]

In [129]:
# Add extra columns with calculated parameters
df['Hansen coordinates']= [[df['dD - Dispersion'].iloc[i], df['dP - Polarity'].iloc[i], df['dH - Hydrogen bonding'].iloc[i]] for i in range(df.shape[0])]
list_GSK_scores = df.columns.values[7:18]
df['Waste'] = (df['Incineration']*df['Recycling']*df['Biotreatment']*df['VOC Emissions'])**0.25
df['Environment']  =(df['Aquatic Impact']*df['Air Impact'])**0.5
df['Health'] = (df['Health Hazard']*df['Exposure Potential'])**0.5
df['Safety'] = (df['Flammability and Explosion']*df['Reactivity and Stability'])**0.5
df['Greenness'] = round((df['Waste']*df['Environment']*df['Health']*df['Safety'])**0.25,2)
df['Ra'] = np.nan

In [130]:
df.head()

,Solvent Name,CAS Number,dD - Dispersion,dP - Polarity,dH - Hydrogen bonding,Melting Point (°C),Boiling Point (°C),Incineration,Recycling,Biotreatment,...,Life Cycle Analysis,Hazard Labels,Precautionary Labels,Hansen coordinates,Waste,Environment,Health,Safety,Greenness,Ra
1,"1,2,4-Trichlorobenzene",120-82-1,20.2,4.2,3.2,17.0,214.0,3.0,7.0,7.0,...,8.0,H302 H315 H331 H400 H410,P261 P264 P270 P271 P273 P280 P301+P312 P302+P...,"[20.2, 4.2, 3.2]",6.191977,3.000000,4.898979,9.486833,5.42,NaN
2,"1,2-Dichlorobenzene (o-DCB)",95-50-1,19.2,6.3,3.3,-17.0,180.0,4.0,8.0,6.0,...,8.0,H302 H315 H317 H319 H332 H335 H400 H410,P261 P264 P270 P271 P272 P273 P280 P301+P312 P...,"[19.2, 6.3, 3.3]",6.054800,2.449490,6.480741,8.944272,5.41,NaN
3,"1,2-Dichloroethane (DCE)",107-06-2,18.0,7.4,4.1,-36.0,84.0,2.0,7.0,5.0,...,7.0,H225 H302 H315 H319 H331 H335 H350,P201 P202 P210 P233 P240 P241 P242 P243 P261 P...,"[18.0, 7.4, 4.1]",4.325308,7.937254,1.414214,7.071068,4.30,NaN
4,"1,2-Dimethoxyethane (Glyme)",110-71-4,15.4,6.3,6.0,-58.0,85.0,4.0,4.0,3.0,...,7.0,H225 H315 H332 H351 H360,P201 P202 P210 P233 P240 P241 P242 P243 P261 P...,"[15.4, 6.3, 6.0]",3.935979,7.483315,2.000000,4.898979,4.12,NaN
5,"1,3-Dioxolane",646-06-0,18.1,6.6,9.3,-95.0,75.0,4.0,4.0,3.0,...,NaN,H225 H319,P210 P233 P240 P241 P242 P243 P264 P280 P303+P...,"[18.1, 6.6, 9.3]",3.722419,5.916080,5.916080,4.242641,4.85,NaN


In [124]:
def update_Ra(coordinates, reference):
    Ra = []
    for value in coordinates:
        # Target vector minus the reference vector
        d = [value[i] - reference[i] for i in range(3)]
        # Square the values to get the hansen index and update the Distance column
        Ra.append(np.sqrt(4*d[0]**2 + d[1]**2 + d[2]**2))
    return Ra

In [125]:
Href = [18,1.5,2]
df['Ra'] = update_distance(df['Hansen coordinates'], Href)

In [114]:
class Solvent(object):
    def __init__(self, parameters):
        # Initalize all the properties of the object from the input information (parameters, a DataFrame structure)
        self.name = parameters['Solvent Name']
        self.CAS = parameters['CAS Number']
        self.melting_point = parameters['Melting Point (°C)']
        self.boiling_point = parameters['Boiling Point (°C)']
        self.dD = parameters['dD - Dispersion']
        self.dP = parameters['dP - Polarity']
        self.dH = parameters['dH - Hydrogen bonding']
        self.hansen_coordinates = [self.dD,self.dP, self.dH]
        self.GSK_scores = dict(Incineration = parameters['Incineration'],\
                              Recycling = parameters['Recycling'])
                                

In [56]:
a = Solvent(df.iloc[0])
a.name
a.CAS
a.hansen_coordinates

[20.2, 4.2, 3.2]

In [47]:
df.iloc[0]

Solvent Name                                             1,2,4-Trichlorobenzene
CAS Number                                                             120-82-1
dD - Dispersion                                                            20.2
dP - Polarity                                                               4.2
dH - Hydrogen bonding                                                       3.2
Melting Point (°C)                                                           17
Boiling Point (°C)                                                          214
Incineration                                                                  3
Recycling                                                                     7
Biotreatment                                                                  7
VOC Emissions                                                                10
Aquatic Impact                                                                1
Air Impact                              

# Plotting

In [131]:
import plotly.graph_objs as go

filt = [True]*df.shape[0]
filt = [df['Greenness'] < 5]
x = df['dD - Dispersion']
y = df['dP - Polarity']
z = df['dH - Hydrogen bonding']

trace = go.Scatter3d(x = x, y = y, z = z, mode='markers', marker=dict(size=8,\
                                                        color = df['Greenness'],\
                                                        colorscale = 'RdYlGn',\
                                                        opacity=0.8,\
                                                        showscale = True),\
                    hovertemplate = '<b>%{text}</b><br>' +\
                                     '%{hovertext}<br>' +\
                                     'dD = %{x:.2f}<br>dP = %{y:.2f}<br>dH = %{z:.2f}',
                    text = df['Solvent Name'],\
                    hovertext = [f'Greenness  = {value:.2f}' for value in df['Greenness']]) 

plot_layout = go.Layout(height=600, title = None,
                paper_bgcolor='white',
                scene={"aspectmode": "cube",
                       "xaxis": {"title": 'dD - Dispersion', },
                       "yaxis": {"title": 'dP - Polarity', },
                       "zaxis": {"title":'dH - Hydrogen bonding' }})

go.Figure(data = trace, layout = layout)

In [105]:
filt

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [106]:
[{'label': i, 'value': i} for i in ['C. Larsen', 'A. Sandström', 'L. Edman', 'etc']]

[{'label': 'C. Larsen', 'value': 'C. Larsen'},
 {'label': 'A. Sandström', 'value': 'A. Sandström'},
 {'label': 'L. Edman', 'value': 'L. Edman'},
 {'label': 'etc', 'value': 'etc'}]

In [142]:
for i in df.index:
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132


In [145]:
df['Solvent Name'].iloc[0]

'1,2,4-Trichlorobenzene'

In [154]:
df.loc[1]

Solvent Name                                             1,2,4-Trichlorobenzene
CAS Number                                                             120-82-1
dD - Dispersion                                                            20.2
dP - Polarity                                                               4.2
dH - Hydrogen bonding                                                       3.2
Melting Point (°C)                                                           17
Boiling Point (°C)                                                          214
Incineration                                                                  3
Recycling                                                                     7
Biotreatment                                                                  7
VOC Emissions                                                                10
Aquatic Impact                                                                1
Air Impact                              

In [170]:
a = [1,3,5,6,7]
b = ['heloo', 'jg','hsh','sh','h']
i = 0
for i,j in zip(b,a):
    print(i, j)

print(j)

heloo 1
jg 3
hsh 5
sh 6
h 7
7


In [174]:
a = [a, 'h', True]

[True, 'h', True]

In [198]:
def say_hi_to(name, age):
    print("We are saying hi to {} who's {}".format(age, name))

In [229]:
class Dog(object):
    """ This classs is for creating a dog"""
    def __init__(self, name):
        """ This initinaluta"""
        self.name = name
        self.owner = None
    def set_owner(self,name):
        """This set sthe owner"""
        self.owner = name
    def change_name(self,name):
        """This set sthe owner"""
        self.name = name

In [230]:
a = Dog('Terry')
b = Dog('Petter')

In [232]:
a.change_name('Bizz')

In [223]:
b.__list_methods()

AttributeError: 'Dog' object has no attribute '__list_methods'

In [233]:
a.name

'Bizz'